In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from gymnasium import Env, spaces
import matplotlib.pyplot as plt

from tiny_shakespeare import TinyShakespeareDataset
from token_level_ppo import TokenLevelPPO
from thoughtsformer_policy import ThoughtsFormerPolicy
from word_embeddings import get_word_embeddings

def token_batched_reshape_with_offset(x: torch.Tensor, max_seq_length: int, thoughts_taken: int) -> torch.Tensor:
    thoughts = thoughts_taken + 1
    max_thoughts = x.size(1) // max_seq_length
    x = x[:,:max_seq_length*thoughts].view(x.size(0), max_seq_length, thoughts)
    return F.pad(x,(0, (max_thoughts - thoughts)))
        
class ThoughtsFormerEnv(Env):
    def __init__(self, vocab_size, max_sequence_length, max_thought_length):
        super(ThoughtsFormerEnv, self).__init__()
        self.vocab_size = vocab_size
        self.max_sequence_length = max_sequence_length 
        self.max_thought_length = max_thought_length
        self.max_context_length = max_sequence_length * (max_thought_length+1)
        
        # Logits
        self.action_space = spaces.MultiDiscrete([vocab_size] * self.max_sequence_length)
 
        
        self.observation_space = spaces.Dict({
            "state" : spaces.MultiDiscrete([vocab_size] * self.max_context_length),
            "thought_step" : spaces.Discrete(max_thought_length+1)
        })
        
        
        
        self.dataset = TinyShakespeareDataset(max_sequence_length,window_offset=max_sequence_length//4)
        self.dataset_len = len(self.dataset)
        self.dataset_iter = 0
        self.thought_step = 0
        

    def reset(self, seed=None):
        super().reset(seed=seed)  # Ensures Gymnasium's seeding is properly handled
        
        self.state, self.labels = self.dataset[self.dataset_iter]
        if self.labels.ndim == 1:
            self.labels = self.labels.unsqueeze(0)
        self.state = F.pad(self.state, (0,self.max_context_length-self.max_sequence_length))
     
        # prepare self.state and massively elongate
        self.dataset_iter += 1
        if self.dataset == self.dataset_len:
            self.dataset_iter = 0

        self.thought_step = 0
        
        obs = {
            'state' : self.state.numpy(),
            'thought_step' : self.thought_step
        }
        return obs, {}
    

    def step(self, action):
        self.state = self.state.view(1,-1)
        sampled_tokens = torch.from_numpy(action.reshape(1,-1))
        
        
        if self.thought_step == self.max_thought_length:
            reward = self.reward(sampled_tokens).flatten() 
            # print(reward.shape)
            done = True
        else:
            reward = torch.zeros(self.max_sequence_length)
            done = False
            # Add the thought!
            self.state = token_batched_reshape_with_offset(self.state, self.max_sequence_length, self.thought_step) # (batch x max_seq_len, (max_thought_len+1))
            # before
            # plt.imshow(self.state[0,:10]); plt.show()
            
            self.state[:,:,self.thought_step+1] = sampled_tokens
            # plt.imshow(self.state[0,:10]); plt.show()
            self.state =  self.state.view(1,-1)
        
        self.state = self.state.view(-1)
        self.thought_step += 1
        obs = {
            'state' : self.state.numpy(),
            'thought_step' : self.thought_step
        }
        
        info = {'reward' : reward} 
        
        # print(info)
        
        return obs, -np.inf, done, False, info #obs, reward, done, truncated, info
    
    def reward(self, tokens: torch.Tensor) -> torch.Tensor:
        a, b = get_word_embeddings(tokens), get_word_embeddings(self.labels)
        x = F.cosine_similarity(a, b, dim=-1) 
        x = torch.maximum(x, torch.Tensor([0]))
        return x.detach()
    # Encourages greater cosine similarity between tokens.
    # def reward(self, action):
    #     return -F.cross_entropy(torch.tensor(action), self.labels, reduction='none')
    
# Check the environment to ensure compatibility

import os


# Now continue with your script

env = ThoughtsFormerEnv(vocab_size=50257, max_sequence_length=256,max_thought_length=3)
policy_kwargs = {
    "max_thought_len" : 3,
    "from_gpt2" : True
}
ppo = TokenLevelPPO(ThoughtsFormerPolicy, env, n_steps=4, batch_size=4, max_sequence_length=256, verbose=2, ent_coef=0.01, policy_kwargs=policy_kwargs)


# ppo.learn(1, log_interval=1)

c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\gym\envs\registration.py:307: DeprecationWarning: The package name gym_minigrid has been deprecated in favor of minigrid. Please uninstall gym_minigrid and install minigrid with `pip install minigrid`. Future releases will be maintained under the new package name minigrid.
  fn()
c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (301966 > 1024). Running this sequence through the model will result in indexing errors


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [3]:
 
policy_kwargs = {
    "max_thought_len" : 1,
    "from_gpt2" : True
}

env = ThoughtsFormerEnv(vocab_size=50257, max_sequence_length=512,max_thought_length=1)
ppo = TokenLevelPPO.load("5600_timesteps.zip", env=env, max_sequence_length=512, verbose=2, ent_coef=0.001,policy_kwargs=policy_kwargs)


c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (301966 > 1024). Running this sequence through the model will result in indexing errors
c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object policy_class. Consider using `custom_objects` argument to replace this object.
Exception: Can't get attribute 'ThoughtsFormerPolicy' on <module 'token_level_ppo' from 'c:\\Users\\bicke\\Documents\\GitHub\\Thoughtsformer\\token_level_ppo.py'>
  warnings.warn(
c:\Us

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cuda device


c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [8]:
s = F.pad(env.dataset[0][0], (0,env.max_context_length-env.max_sequence_length))

obs = {
    'state' : s.reshape(1,-1).to('cuda'),
    'thought_step' : 0

}
sampled_tokens, final_logits = ppo.policy.model.entire_thought_generation(obs['state'], torch.zeros_like(s).view(1,-1).to('cuda'))

from transformers import GPT2Tokenizer
p = GPT2Tokenizer.from_pretrained("gpt2")

y = ppo.policy.model._sample_tokens(final_logits)[0]

def process(s):
  return str(s).replace("\n", "\\n").strip()
print(f"{str('Original Token'):13} | {str('Intermediate Thought'):16} | Prediction")
print("---------------------------------------------------")
for i in range(100):
  print(f"""\
{process(p.decode(sampled_tokens[0][2*i])):13}  |  {process(p.decode(sampled_tokens[0][2*i+1])):19} |   {process(p.decode(y[0][i])):10}""")
  
  

Original Token | Intermediate Thought | Prediction
---------------------------------------------------
First          |  .                   |   \n        
Citizen        |  duct                |   .         
:              |  \n                  |   \n        
\n             |  Gamb                |   is        
Before         |  ello                |   the       
we             |  Grab                |   have      
proceed        |  gee                 |   .         
any            |  iman                |   This      
further        |  genital             |   ,         
,              |  benefits            |   as        
hear           |  ance                |   it        
me             |  puberty             |   have      
speak          |  am                  |   .         
.              |  032                 |   \n        
\n             |  outsourcing         |   \n        
\n             |  pulling             |   We        
All            |  ponies              |   is     

In [30]:
x = TinyShakespeareDataset(512,1024*4)

c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (301966 > 1024). Running this sequence through the model will result in indexing errors


In [31]:
obs = {'state' : x[0][0].view(1,-1).to('cuda'), 'thought_step' : 1}
a,b = ppo.policy.forward(obs)

In [39]:
n = F.softmax(a,dim=-1)
n[:,:,n.argmax(dim=-1).flatten()]

tensor([[[0.0055, 0.0048, 0.0055,  ..., 0.0021, 0.0021, 0.0021],
         [0.0047, 0.0052, 0.0047,  ..., 0.0024, 0.0024, 0.0024],
         [0.0053, 0.0051, 0.0053,  ..., 0.0027, 0.0027, 0.0027],
         ...,
         [0.0104, 0.0040, 0.0104,  ..., 0.0233, 0.0233, 0.0233],
         [0.0106, 0.0039, 0.0106,  ..., 0.0228, 0.0228, 0.0228],
         [0.0110, 0.0038, 0.0110,  ..., 0.0236, 0.0236, 0.0236]]],
       device='cuda:0', grad_fn=<IndexBackward0>)

In [43]:
from transformers import GPT2Tokenizer
p = GPT2Tokenizer.from_pretrained("gpt2")

c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [45]:
p.decode(x)

','

In [27]:
logger.name_to_value

defaultdict(float, {})

In [2]:
import torch
import torch.nn as nn
from thoughtsformer import ThoughtsFormer, simple_batched_reshape_with_offset
from stable_baselines3.common.policies import ActorCriticPolicy

class _ThoughtsFormerPolicy(ActorCriticPolicy):
    def __init__(self):
        self.model = ThoughtsFormer.from_pretrained_GPT2()
        
    def forward(self, obs: dict):
        assert 'state' in obs and 'thought_step' in obs, f"'state' and 'thought_step' should be keys of the observation dictionary"
        return self.model.forward_ppo_with_tokens(obs['state'], torch.zeros_like(obs['state']), obs['thought_step'])
    
    def _get_action_dist_from_obs(self, obs):
        return self.forward(obs)[0]

    def _get_value_from_obs(self, obs):
        return self.forward(obs)[1]
    
env = ThoughtsFormerEnv()

TypeError: ThoughtsFormerEnv.__init__() missing 3 required positional arguments: 'vocab_size', 'max_sequence_length', and 'max_thought_length'

In [5]:

# Check the environment to ensure compatibility
from token_level_ppo import TokenLevelPPO, TokenLevelRolloutBuffer, ThoughtsFormerPolicy
ppo = TokenLevelPPO(ThoughtsFormerPolicy, env, n_steps=2, batch_size=2, max_sequence_length=512, verbose=2)


state torch.Size([11264])


TypeError: TokenLevelPPO.collect_rollouts() got an unexpected keyword argument 'n_rollout_steps'

In [13]:
import torch.nn.functional as F
x = torch.tensor([1,2,3,4,0,0,0,0,0,0,0,0,0,0,0,0]).view(1,-1)


def simple_batched_reshape_with_offset(x: torch.Tensor, max_seq_length: int, thoughts_taken: int) -> torch.Tensor:
    thoughts = thoughts_taken + 1
    max_thoughts = x.size(1) // max_seq_length
    x = x[:,:max_seq_length*thoughts].view(x.size(0), max_seq_length, thoughts)
    return F.pad(x,(0, (max_thoughts - thoughts)))

simple_batched_reshape_with_offset(x,4,1)

tensor([[[1, 2, 0, 0],
         [3, 4, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]])

In [9]:
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from gymnasium import Env, spaces

class CustomEnv(Env):
    def __init__(self):
        super(CustomEnv, self).__init__()
        # Define action and observation space
        self.action_space = spaces.Discrete(4)  # 4 actions (up, right, down, left)
        self.observation_space = spaces.MultiDiscrete([3, 4])  # (y, x) coordinates
        self.state = (2, 0)  # Initial state

    def reset(self, seed=None):
        super().reset(seed=seed)  # Ensures Gymnasium's seeding is properly handled
        """Resets the environment to the initial state."""
        self.state = (2, 0)
        return np.array(self.state), {}

    def step(self, action):
        """Take an action in the environment."""
        y, x = self.state

        if action == 0:  # up
            y += 1
        elif action == 1:  # right
            x += 1
        elif action == 2:  # down
            y -= 1
        elif action == 3:  # left
            x -= 1

        # Keep coordinates within bounds
        y, x = max(0, min(2, y)), max(0, min(3, x))
        self.state = (y, x)

        # Compute reward and done
        reward, done = self.reward(self.state
                                   )
        return np.array(self.state), reward, done, False, {}

    def reward(self, state):
        if state == (0, 3):
            return 1, True  # Goal state with reward
        elif state == (1, 3):
            return -1, True  # Failure state with negative reward
        else:
            return 0, False  # No reward, episode continues

# Check the environment to ensure compatibility
env = CustomEnv()
check_env(env)

# Create the PPO model
model = PPO("MlpPolicy", env, verbose=1)

# Train the model
model.learn(total_timesteps=1000)

# Test the trained model
obs = env.reset()[0]
done = False
while not done:
    action, _states = model.predict(obs)
    obs, reward, done, _, info = env.step(action)
    print(f"State: {obs}, Reward: {reward}")

env.close()


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 31.4     |
|    ep_rew_mean     | -0.292   |
| time/              |          |
|    fps             | 327      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
State: [2 1], Reward: 0
State: [1 1], Reward: 0
State: [1 2], Reward: 0
State: [1 1], Reward: 0
State: [1 0], Reward: 0
State: [0 0], Reward: 0
State: [0 1], Reward: 0
State: [0 1], Reward: 0
State: [0 1], Reward: 0
State: [0 0], Reward: 0
State: [0 0], Reward: 0
State: [0 0], Reward: 0
State: [0 0], Reward: 0
State: [0 0], Reward: 0
State: [0 1], Reward: 0
State: [0 1], Reward: 0
State: [0 2], Reward: 0
State: [0 1], Reward: 0
State: [0 2], Reward: 0
State: [0 1], Reward: 0
State: [1 1], Reward: 0
State: [1 2], Reward: 0
State: [0 2], Reward: 0
State: [1 2],

In [110]:
value = np.zeros((3,4))
policy = np.zeros((3,4))

In [ ]:
import numpy as np
from enum import Enum
class Actions(Enum):
    up = 0,
    right = 1,
    down = 2,
    left  = 3,
    none = -1

class env():
    
    actions = (Actions.up, Actions.right, Actions.down, Actions.left)
    states = (
            ((0,0), (0,1), (0,2), (0,3)),
            ((1,0), (1,1), (1,2), (1,3)),
            ((2,0), (2,1), (2,2), (2,3))
    )
    
    iter_states = (
            (0,0), (0,1), (0,2), (0,3),
            (1,0), (1,1), (1,2), (1,3),
            (2,0), (2,1), (2,2), (2,3)
    )
    
    def __init__(self):
        pass
    def step(self, state, action):
        y, x = state
        
        if action == Actions.up:
            y += 1
        if action == Actions.right:
            x += 1
        if action == Actions.down:
            y -= 1
        if action == Actions.left:
            x -= 1

        y, x = max(y,0), max(x,0)
        y, x = min(y,2), min(x,3)
        state_next = (y,x)
        reward, done = self.reward((y,x))
        return state_next, reward, done
        
    def reward(self, next_state):
        if next_state == (0,3):
            return 1, True
        elif next_state == (1,3):
            return -1, True
        else:
            return 0, False

In [113]:
H = 3
gamma = 0.9
e = env()
for i in range(H):
    for state in env.iter_states:
        
        if state == (0, 3) or state == (1, 3):
            continue
        
        if state == (1, 1):
            continue
        
        max_reward = 0
        p = Actions.none.value
        for idx, action in enumerate(env.actions):
            n_state, rew = e.step(state, action)
            n = rew + gamma * value[n_state]
            if n > max_reward:
                max_reward = n
                p = action.value[0]
        policy[state] = p
        value[state] = max_reward

In [114]:
value

array([[0.81  , 0.9   , 1.    , 0.    ],
       [0.729 , 0.    , 0.9   , 0.    ],
       [0.6561, 0.729 , 0.81  , 0.729 ]])

In [104]:
policy

class Actions(Enum):
    down = 0,
    right = 1,
    up = 2,
    left  = 3,
    none = -1


array([[1., 1., 1., 0.],
       [2., 0., 2., 0.],
       [1., 1., 2., 3.]])

In [ ]:

# Janky token_level_ppo load in for when the environment and the previous weights dont match

import warnings
from stable_baselines3 import PPO

from stable_baselines3.common.buffers import DictRolloutBuffer
import torch.nn.functional as F
import numpy as np
from gymnasium import Env, spaces

import torch
from stable_baselines3.common.utils import obs_as_tensor
from torch.distributions import Categorical



class TokenLevelRolloutBuffer(DictRolloutBuffer):
    def __init__(self, *args, max_sequence_length: int, **kwargs) -> None:
        self.max_sequence_length = max_sequence_length
        super().__init__(*args, **kwargs)
        
    def reset(self) -> None:
        super().reset()
        self.rewards = np.zeros((self.buffer_size, self.n_envs, self.max_sequence_length), dtype=np.float32)
        self.values = np.zeros_like(self.rewards)
        self.actions = np.zeros_like(self.rewards) 
        self.advantages = np.zeros_like(self.rewards)
        self.log_probs = np.zeros_like(self.rewards)
    def add(  # type: ignore[override]
        self,
        obs: dict[str, np.ndarray],
        action: np.ndarray,
        reward: np.ndarray,
        episode_start: np.ndarray,
        value: torch.Tensor,
        log_prob: torch.Tensor,
    ) -> None:
        """
        :param obs: Observation
        :param action: Action
        :param reward:
        :param episode_start: Start of episode signal.
        :param value: estimated value of the current state
            following the current policy.
        :param log_prob: log probability of the action
            following the current policy.
        """
        if len(log_prob.shape) == 0:
            # Reshape 0-d tensor to avoid error
            log_prob = log_prob.reshape(-1, 1)

        for key in self.observations.keys():
            obs_ = np.array(obs[key])
            # Reshape needed when using multiple envs with discrete observations
            # as numpy cannot broadcast (n_discrete,) to (n_discrete, 1)
            if isinstance(self.observation_space.spaces[key], spaces.Discrete):
                obs_ = obs_.reshape((self.n_envs,) + self.obs_shape[key])
            self.observations[key][self.pos] = obs_

        # Reshape to handle multi-dim and discrete action spaces, see GH #970 #1392
        action = action.reshape((self.n_envs, self.max_sequence_length)) # CHANGE IN EMMETT'S CODE (kinda janky change)
        log_prob = log_prob.reshape((self.n_envs, self.action_dim)) # CHANGE IN EMMETT'S CODE 
        self.actions[self.pos] = np.array(action)
        self.rewards[self.pos] = np.array(reward)
        self.episode_starts[self.pos] = np.array(episode_start)
        self.values[self.pos] = value.clone().cpu().numpy().flatten()
        self.log_probs[self.pos] = log_prob.clone().cpu().numpy() # CHANGE IN EMMETT'S CODE 
        self.pos += 1
        if self.pos == self.buffer_size:
            self.full = True

    def compute_returns_and_advantage(self, last_values: torch.Tensor | np.ndarray, dones: np.ndarray) -> None:
        """
        Adapted for token-level GAE
        """
        if isinstance(last_values, torch.Tensor):
            last_values = last_values.cpu().numpy() 
        last_gae_lam = 0
        for step in reversed(range(self.buffer_size)):
            if step == self.buffer_size - 1:
                next_non_terminal = 1.0 - dones
                next_values = last_values
            else:
                next_non_terminal = 1.0 - self.episode_starts[step + 1]
                next_values = self.values[step + 1]
            
            delta = self.rewards[step] + self.gamma * next_values * next_non_terminal - self.values[step]
            last_gae_lam = delta + self.gamma * self.gae_lambda * next_non_terminal * last_gae_lam
            self.advantages[step] = last_gae_lam

        # Reshape advantages and returns to match token-level structure
        self.advantages = self.advantages.reshape(self.buffer_size, -1, self.max_sequence_length)
        self.returns = self.advantages + self.values

class TokenLevelPPO(PPO):
    def __init__(self, policy, env, max_sequence_length, **kwargs):
        self.max_sequence_length = max_sequence_length
        super().__init__(policy, env, **kwargs)
        self.rollout_buffer = TokenLevelRolloutBuffer(
            self.n_steps,
            self.observation_space,
            self.action_space,
            max_sequence_length=max_sequence_length,
            device=self.device,
            gae_lambda=self.gae_lambda,
            gamma=self.gamma,
            n_envs=self.n_envs,
        )
        
    
    @classmethod
    def load(  # noqa: C901
        cls,
        path,
        max_sequence_length,
        env = None,
        device = "auto",
        custom_objects = None,
        print_system_info = False,
        force_reset = True,
        **kwargs,
    ):
        from stable_baselines3.common.preprocessing import check_for_nested_spaces, is_image_space, is_image_space_channels_first
        from stable_baselines3.common.save_util import load_from_zip_file, recursive_getattr, recursive_setattr, save_to_zip_file
        from stable_baselines3.common.vec_env.patch_gym import _convert_space, _patch_env
        from stable_baselines3.common.utils import (
            check_for_correct_spaces,
            get_device,
            get_schedule_fn,
            get_system_info,
            set_random_seed,
            update_learning_rate,
        )
        import warnings
        """
        Load the model from a zip-file.
        Warning: ``load`` re-creates the model from scratch, it does not update it in-place!
        For an in-place load use ``set_parameters`` instead.

        :param path: path to the file (or a file-like) where to
            load the agent from
        :param env: the new environment to run the loaded model on
            (can be None if you only need prediction from a trained model) has priority over any saved environment
        :param device: Device on which the code should run.
        :param custom_objects: Dictionary of objects to replace
            upon loading. If a variable is present in this dictionary as a
            key, it will not be deserialized and the corresponding item
            will be used instead. Similar to custom_objects in
            ``keras.models.load_model``. Useful when you have an object in
            file that can not be deserialized.
        :param print_system_info: Whether to print system info from the saved model
            and the current system info (useful to debug loading issues)
        :param force_reset: Force call to ``reset()`` before training
            to avoid unexpected behavior.
            See https://github.com/DLR-RM/stable-baselines3/issues/597
        :param kwargs: extra arguments to change the model when loading
        :return: new model instance with loaded parameters
        """
        if print_system_info:
            print("== CURRENT SYSTEM INFO ==")
            get_system_info()

        data, params, pytorch_variables = load_from_zip_file(
            path,
            device=device,
            custom_objects=custom_objects,
            print_system_info=print_system_info,
        )

        assert data is not None, "No data found in the saved file"
        assert params is not None, "No params found in the saved file"

        # Remove stored device information and replace with ours
        if "policy_kwargs" in data:
            if "device" in data["policy_kwargs"]:
                del data["policy_kwargs"]["device"]
            # backward compatibility, convert to new format
            if "net_arch" in data["policy_kwargs"] and len(data["policy_kwargs"]["net_arch"]) > 0:
                saved_net_arch = data["policy_kwargs"]["net_arch"]
                if isinstance(saved_net_arch, list) and isinstance(saved_net_arch[0], dict):
                    data["policy_kwargs"]["net_arch"] = saved_net_arch[0]

        if "policy_kwargs" in kwargs and kwargs["policy_kwargs"] != data["policy_kwargs"]:
            pass
            # raise ValueError(
            #     f"The specified policy kwargs do not equal the stored policy kwargs."
            #     f"Stored kwargs: {data['policy_kwargs']}, specified kwargs: {kwargs['policy_kwargs']}"
            # )

        if "observation_space" not in data or "action_space" not in data:
            raise KeyError("The observation_space and action_space were not given, can't verify new environments")

        # Gym -> Gymnasium space conversion
        for key in {"observation_space", "action_space"}:
            data[key] = _convert_space(data[key])

        if env is not None:
            # Wrap first if needed
            env = cls._wrap_env(env, data["verbose"])
            # Check if given env is valid
            check_for_correct_spaces(env, data["observation_space"], data["action_space"])
            # Discard `_last_obs`, this will force the env to reset before training
            # See issue https://github.com/DLR-RM/stable-baselines3/issues/597
            if force_reset and data is not None:
                data["_last_obs"] = None
            # `n_envs` must be updated. See issue https://github.com/DLR-RM/stable-baselines3/issues/1018
            if data is not None:
                data["n_envs"] = env.num_envs
        else:
            # Use stored env, if one exists. If not, continue as is (can be used for predict)
            if "env" in data:
                env = data["env"]
        from thoughtsformer_policy import ThoughtsFormerPolicy
        model = cls(
            policy=ThoughtsFormerPolicy,
            env=env,
            device=device,
            max_sequence_length=max_sequence_length,
            _init_setup_model=False,  # type: ignore[call-arg]
            **kwargs
        )

        # load parameters
        model.__dict__.update(data)
        model.__dict__.update(kwargs)
        model._setup_model()

        try:
            # put state_dicts back in place
            model.set_parameters(params, exact_match=True, device=device)
        except RuntimeError as e:
            # Patch to load Policy saved using SB3 < 1.7.0
            # the error is probably due to old policy being loaded
            # See https://github.com/DLR-RM/stable-baselines3/issues/1233
            if "pi_features_extractor" in str(e) and "Missing key(s) in state_dict" in str(e):
                model.set_parameters(params, exact_match=False, device=device)
                warnings.warn(
                    "You are probably loading a model saved with SB3 < 1.7.0, "
                    "we deactivated exact_match so you can save the model "
                    "again to avoid issues in the future "
                    "(see https://github.com/DLR-RM/stable-baselines3/issues/1233 for more info). "
                    f"Original error: {e} \n"
                    "Note: the model should still work fine, this only a warning."
                )
            else:
                raise e
        # put other pytorch variables back in place
        if pytorch_variables is not None:
            for name in pytorch_variables:
                # Skip if PyTorch variable was not defined (to ensure backward compatibility).
                # This happens when using SAC/TQC.
                # SAC has an entropy coefficient which can be fixed or optimized.
                # If it is optimized, an additional PyTorch variable `log_ent_coef` is defined,
                # otherwise it is initialized to `None`.
                if pytorch_variables[name] is None:
                    continue
                # Set the data attribute directly to avoid issue when using optimizers
                # See https://github.com/DLR-RM/stable-baselines3/issues/391
                recursive_setattr(model, f"{name}.data", pytorch_variables[name].data)

        # Sample gSDE exploration matrix, so it uses the right device
        # see issue #44
        if model.use_sde:
            model.policy.reset_noise()  # type: ignore[operator]
            
            
        
        return model
    
    def _setup_model(self) -> None:
        super()._setup_model()
        self.rollout_buffer = TokenLevelRolloutBuffer(
            self.n_steps,
            self.observation_space,
            self.action_space,
            max_sequence_length=self.max_sequence_length,
            device=self.device,
            gae_lambda=self.gae_lambda,
            gamma=self.gamma,
            n_envs=self.n_envs,
        )
        

    def collect_rollouts(self, env, callback, rollout_buffer, n_rollout_steps):
        # Reset or initialize variables
        assert self._last_obs is not None, "No previous observation was provided"
        # Switch to eval mode (this affects batch norm / dropout)
        self.policy.set_training_mode(False)
        
        rollout_buffer.reset()
        for step in range(n_rollout_steps):
            # Get actions and values from the policy
            with torch.no_grad():
                obs_tensor = obs_as_tensor(self._last_obs, self.device)
                actions, values, log_probs = self.policy(obs_tensor)
            actions = actions.cpu().numpy()


            if isinstance(self.action_space, spaces.Box):
                if self.policy.squash_output:
                    # Unscale the actions to match env bounds
                    # if they were previously squashed (scaled in [-1, 1])
                    clipped_actions = self.policy.unscale_action(clipped_actions)
                else:
                    # Otherwise, clip the actions to avoid out of bound error
                    # as we are sampling from an unbounded Gaussian distribution
                    clipped_actions = np.clip(actions, self.action_space.low, self.action_space.high)

            new_observations, rewards, dones, infos = env.step(actions)
            rewards = torch.stack([infos[idx]['reward'] for idx in range(len(infos))]).numpy()
            # actions = torch.stack([infos[idx]['actions_taken'] for idx in range(len(infos))]).numpy()
            
            self.num_timesteps += env.num_envs
            
            self._update_info_buffer(infos, dones)
            
            
            for idx, done in enumerate(dones):
                if (
                    done
                    and infos[idx].get("terminal_observation") is not None
                    and infos[idx].get("TimeLimit.truncated", False)
                ):
                    terminal_obs = self.policy.obs_to_tensor(infos[idx]["terminal_observation"])[0]
                    with torch.no_grad():
                        terminal_value = self.policy.predict_values(terminal_obs)[0]  # type: ignore[arg-type]
                    rewards[idx] += self.gamma * terminal_value

  
            
            # Store data in the buffer
            
            rollout_buffer.add(
                self._last_obs,  # type: ignore[arg-type]
                actions,
                rewards,
                self._last_episode_starts,  # type: ignore[arg-type]
                values,
                log_probs)
            
            self._last_obs = new_observations  
            self._last_episode_starts = dones
            
        with torch.no_grad():
            # Compute value for the last timestep
            values = self.policy.predict_values(obs_as_tensor(new_observations, self.device))  # type: ignore[arg-type]
            
        rollout_buffer.compute_returns_and_advantage(last_values=values, dones=dones)
        return True
        
    def train(self) -> None:
        counter = 0
        for epoch in range(self.n_epochs):
            for rollout_data in self.rollout_buffer.get(self.batch_size):
                
                with torch.autograd.set_detect_anomaly(True):
                    actions = rollout_data.actions
                    actions = actions.reshape(actions.size(0), self.max_sequence_length).to(torch.long)
                    # actions = actions.reshape(actions.size(0),self.
                    rollout_data.observations['state'] = rollout_data.observations['state'].to(torch.long)
                    # rollout_data.observations['thought_step'] 
                    values, action_log_prob, entropy = self.policy.evaluate_actions(rollout_data.observations, actions)

                    values = values.view(-1, actions.shape[1]) # (batch x tokens)
                    # Normalize advantage
                    advantages = rollout_data.advantages
                    advantages = advantages.view(-1, actions.shape[1]) # (batch x tokens)
                    if advantages.size(0) >= 3: # Only normalize if there's a sizable batch
                        advantages = (advantages - advantages.mean(dim=0, keepdim=True)) / (advantages.std(dim=0, keepdim=True) + 1e-8)
                    else:
                        warnings.warn("Size of current batch is less than or equal to two, so advantage batch normalization has been disabled this iteration.")

                    
                    # ratio between old and new policy, should be one at the first iteration
                
                    action_log_prob = action_log_prob.view(-1, actions.shape[1]) # what our policy thinks about the old action taken
                    old_log_prob = rollout_data.old_log_prob.view(-1, actions.shape[1]) # what the old policy thinks about the action taken
                    # old_log_prob = F.log_softmax(old_log_prob, dim=-1)
                    # old_log_prob = old_log_prob.gather(-1, actions.unsqueeze(-1)).squeeze(-1) # FIX: Terrible extraction of old log prob for taken actions after the fact and requires storing an outrageous amount of information
                    ratio = torch.exp(action_log_prob - old_log_prob)
                    
                    current_clip_range = self.clip_range(counter / self._total_timesteps)
                    # clipped surrogate loss
                    policy_loss_1 = advantages * ratio
                    policy_loss_2 = advantages * torch.clamp(ratio, 1 - current_clip_range, 1 + current_clip_range)
                    policy_loss = -torch.min(policy_loss_1, policy_loss_2)

                    
                    # Value clipping is omitted in the token-based version, but could be added if needed
                    value_loss = F.mse_loss(rollout_data.returns.view(-1, actions.shape[1]), values, reduction='none')
                    # Value loss using the TD(gae_lambda) target

                    # Entropy loss favor exploration
                    if entropy is None:
                        entropy_loss = -action_log_prob
                    else:
                        entropy_loss = -entropy.view(-1, actions.shape[1])

                    token_loss = policy_loss + self.vf_coef * value_loss + self.ent_coef * entropy_loss
                    loss = token_loss.mean()
                    
                    self.policy.optimizer.zero_grad()
                
                    loss.backward()
                # .backward()
                # Clip grad norm
                torch.nn.utils.clip_grad_norm_(self.policy.parameters(), self.max_grad_norm)
                self.policy.optimizer.step()
               
                # Logging
                counter+=1
            
                self.logger.record("train/loss", loss.item())
                self.logger.record("train/policy_loss", policy_loss.mean().item())
                self.logger.record("train/value_loss", value_loss.mean().item())
                self.logger.record("train/entropy_loss", entropy_loss.mean().item())
                self.logger.record("train/mean_reward", rollout_data.returns.mean().item())
                
        self.logger.dump(step=counter)